In [1]:
import face_recognition
from face_recognition import face_locations
import os
import pandas as pd
import h5py
from tqdm import tqdm
from recursive_dict import *
import scipy.io
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import pickle
import numpy as np

In [2]:

loaded_model = pickle.load(open('model.pkl', 'rb'))


In [3]:
loaded_model

MLPClassifier(hidden_layer_sizes=(128, 128), max_iter=5000, verbose=True)

In [4]:
# set data paths
data_dir = '/Users/arthur/Desktop/2022 EM/Machine Learning/Projet/DB + Model'
img_dir = os.path.join(data_dir,'df_2')
indices_path = os.path.join(data_dir, 'indices_train_test.mat')
attr_path = os.path.join(data_dir, 'lfw_att_73.mat')

In [5]:

attr_name_mat = '/Users/arthur/Desktop/2022 EM/Machine Learning/Projet/DB + Model/LWFA+/attrname.mat'
attr_name = scipy.io.loadmat(attr_name_mat)['AttrName']
attr_name = [str(s[0]) for s in attr_name.tolist()[0]]

In [6]:
FichList = [ f for f in os.listdir('/Users/arthur/Desktop/2022 EM/Machine Learning/Projet/DB + Model/df_2') if os.path.isfile(os.path.join('/Users/arthur/Desktop/2022 EM/Machine Learning/Projet/DB + Model/df_2',f)) ]
print(FichList)

['images (2).jpeg', 'images (4).jpeg', 'téléchargement (8).jpeg', 'images.jpeg', 'téléchargement (10).jpeg', 'images (6).jpeg', 'images (1).jpeg', 'images (3).jpeg', 'téléchargement.jpeg', 'images (7).jpeg', 'images (5).jpeg', 'téléchargement (9).jpeg', 'téléchargement (11).jpeg', 'téléchargement (5).jpeg', 'téléchargement (7).jpeg', 'téléchargement (3).jpeg', 'téléchargement (1).jpeg', 'téléchargement (6).jpeg', 'Margot_Robbie.jpeg', 'téléchargement (4).jpeg', 'images (8).jpeg', 'téléchargement (2).jpeg']


In [7]:
df_label = pd.DataFrame(columns=attr_name, index= FichList)

In [8]:
df_label

,Male,Asian,White,Black,Baby,Child,Youth,Middle Aged,Senior,Black Hair,...,Pale Skin,5 o Clock Shadow,Strong Nose-Mouth Lines,Wearing Lipstick,Flushed Face,High Cheekbones,Brown Eyes,Wearing Earrings,Wearing Necktie,Wearing Necklace
images (2).jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
images (4).jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
téléchargement (8).jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
images.jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
téléchargement (10).jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
images (6).jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
images (1).jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
images (3).jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
téléchargement.jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
images (7).jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_label.index

Index(['images (2).jpeg', 'images (4).jpeg', 'téléchargement (8).jpeg',
       'images.jpeg', 'téléchargement (10).jpeg', 'images (6).jpeg',
       'images (1).jpeg', 'images (3).jpeg', 'téléchargement.jpeg',
       'images (7).jpeg', 'images (5).jpeg', 'téléchargement (9).jpeg',
       'téléchargement (11).jpeg', 'téléchargement (5).jpeg',
       'téléchargement (7).jpeg', 'téléchargement (3).jpeg',
       'téléchargement (1).jpeg', 'téléchargement (6).jpeg',
       'Margot_Robbie.jpeg', 'téléchargement (4).jpeg', 'images (8).jpeg',
       'téléchargement (2).jpeg'],
      dtype='object')

In [10]:
vecs = []
fnames = []
i = 0
for fname in tqdm(df_label.index):
    i += 1
    img_path = os.path.join(img_dir, fname)
    X_img = face_recognition.load_image_file(img_path)
    X_faces_loc = face_recognition.face_locations(X_img)
    if len(X_faces_loc) != 1:
        continue
    faces_encoding = face_recognition.face_encodings(X_img, known_face_locations=X_faces_loc)[0]
    
    vecs.append(faces_encoding)
    fnames.append(fname)
    
df_feat = pd.DataFrame(vecs, index=fnames)
df_label = df_label[df_label.index.isin(df_feat.index)]
df_feat.sort_index(inplace=True)
df_label.sort_index(inplace=True)

100%|███████████████████████████████████████████| 22/22 [00:01<00:00, 12.53it/s]
/var/folders/h5/qcn9lt4953j5p0hr89gbvv3r0000gn/T/ipykernel_22895/3413094446.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_label.sort_index(inplace=True)


'Margot_Robbie.jpeg'

In [11]:
img_path

'/Users/arthur/Desktop/2022 EM/Machine Learning/Projet/DB + Model/df_2/téléchargement (2).jpeg'

In [12]:
fnames

['images (2).jpeg',
 'images (4).jpeg',
 'téléchargement (8).jpeg',
 'images.jpeg',
 'téléchargement (10).jpeg',
 'images (1).jpeg',
 'images (3).jpeg',
 'téléchargement.jpeg',
 'images (7).jpeg',
 'images (5).jpeg',
 'téléchargement (9).jpeg',
 'téléchargement (11).jpeg',
 'téléchargement (5).jpeg',
 'téléchargement (3).jpeg',
 'téléchargement (1).jpeg',
 'téléchargement (6).jpeg',
 'Margot_Robbie.jpeg',
 'téléchargement (4).jpeg',
 'images (8).jpeg',
 'téléchargement (2).jpeg']

In [13]:
predict_score = loaded_model.predict_proba(vecs)

/opt/anaconda3/envs/emlyon/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


In [14]:
df_2 = pd.DataFrame(predict_score, columns=df_label.columns, index=fnames)

In [15]:
df_2

,Male,Asian,White,Black,Baby,Child,Youth,Middle Aged,Senior,Black Hair,...,Pale Skin,5 o Clock Shadow,Strong Nose-Mouth Lines,Wearing Lipstick,Flushed Face,High Cheekbones,Brown Eyes,Wearing Earrings,Wearing Necktie,Wearing Necklace
images (2).jpeg,0.048043,0.004577,0.700706,0.018557,0.000221,0.006466,0.934794,0.016041,0.001084,0.076571,...,0.521703,0.274373,0.064675,0.989899,0.113123,0.236688,0.436460,0.907652,0.087476,0.907671
images (4).jpeg,0.003665,0.010404,0.926299,0.000687,0.009528,0.039097,0.903524,0.003452,0.000365,0.009632,...,0.749477,0.004266,0.786746,0.984396,0.095292,0.889437,0.385233,0.937880,0.084803,0.969196
téléchargement (8).jpeg,0.034727,0.002825,0.990861,0.005139,0.000127,0.077490,0.793430,0.010132,0.003408,0.007951,...,0.878210,0.022837,0.034530,0.978536,0.009900,0.187859,0.050397,0.805389,0.057698,0.863229
images.jpeg,0.085228,0.000890,0.953468,0.000429,0.002975,0.183429,0.533348,0.020205,0.003021,0.014423,...,0.785971,0.056653,0.171001,0.771515,0.056092,0.118686,0.091946,0.366086,0.079471,0.763487
téléchargement (10).jpeg,0.124518,0.042272,0.863621,0.011513,0.001376,0.093295,0.869440,0.014964,0.001034,0.050097,...,0.611517,0.019239,0.743001,0.843819,0.091223,0.911107,0.800959,0.448060,0.091307,0.804841
images (1).jpeg,0.144862,0.000135,0.991073,0.006559,0.000034,0.043741,0.322398,0.002492,0.181365,0.001887,...,0.315935,0.005825,0.878647,0.792028,0.112157,0.942432,0.619188,0.204538,0.045956,0.845506
images (3).jpeg,0.037411,0.006309,0.556520,0.012890,0.014843,0.043165,0.884001,0.006310,0.000713,0.214667,...,0.608222,0.113202,0.201700,0.931727,0.034023,0.218860,0.468016,0.710759,0.050870,0.709083
téléchargement.jpeg,0.137742,0.000311,0.987634,0.003268,0.001137,0.074307,0.353062,0.007015,0.004013,0.007093,...,0.463794,0.062280,0.783707,0.221592,0.073193,0.688171,0.466389,0.137944,0.191729,0.692180
images (7).jpeg,0.004632,0.078940,0.347111,0.036489,0.005255,0.114993,0.934409,0.001218,0.000319,0.275476,...,0.581578,0.010212,0.249697,0.980623,0.022204,0.521147,0.573849,0.964266,0.017924,0.797512
images (5).jpeg,0.027156,0.020098,0.786079,0.010306,0.000186,0.103337,0.902819,0.002375,0.001972,0.020407,...,0.618852,0.007228,0.248929,0.955735,0.012407,0.713120,0.562786,0.914198,0.033391,0.917045


In [16]:
df_2['Female'] = 1 - df_2['Male']
    

In [17]:
df_score = df_2[['Male','Female', 'Asian', 'Black', 'White','Indian','Black Hair','Blond Hair','Brown Hair','Curly Hair','Wavy Hair','Straight Hair','Oval Face','Square Face','Round Face','Baby', 'Child', 'Youth', 'Middle Aged', 'Senior']]

In [18]:
df_score['Gender'] = df_score[['Male', 'Female']].idxmax(axis=1)

/var/folders/h5/qcn9lt4953j5p0hr89gbvv3r0000gn/T/ipykernel_22895/2760167194.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_score['Gender'] = df_score[['Male', 'Female']].idxmax(axis=1)


In [19]:
df_score['Race'] = df_score[['Asian', 'Black', 'White','Indian']].idxmax(axis=1)

/var/folders/h5/qcn9lt4953j5p0hr89gbvv3r0000gn/T/ipykernel_22895/1693712476.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_score['Race'] = df_score[['Asian', 'Black', 'White','Indian']].idxmax(axis=1)


In [20]:
df_score['Hair color'] = df_score[['Black Hair','Blond Hair','Brown Hair']].idxmax(axis=1)

/var/folders/h5/qcn9lt4953j5p0hr89gbvv3r0000gn/T/ipykernel_22895/3021939245.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_score['Hair color'] = df_score[['Black Hair','Blond Hair','Brown Hair']].idxmax(axis=1)


In [21]:
df_score['Hair Shape'] = df_score[['Curly Hair','Wavy Hair','Straight Hair']].idxmax(axis=1)

/var/folders/h5/qcn9lt4953j5p0hr89gbvv3r0000gn/T/ipykernel_22895/3217974210.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_score['Hair Shape'] = df_score[['Curly Hair','Wavy Hair','Straight Hair']].idxmax(axis=1)


In [22]:
df_score['Face Shape'] = df_score[['Oval Face','Square Face','Round Face']].idxmax(axis=1)

/var/folders/h5/qcn9lt4953j5p0hr89gbvv3r0000gn/T/ipykernel_22895/1219207373.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_score['Face Shape'] = df_score[['Oval Face','Square Face','Round Face']].idxmax(axis=1)


In [23]:
df_score['Looking Age'] = df_score[['Baby', 'Child', 'Youth', 'Middle Aged', 'Senior']].idxmax(axis=1)

/var/folders/h5/qcn9lt4953j5p0hr89gbvv3r0000gn/T/ipykernel_22895/2682423066.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_score['Looking Age'] = df_score[['Baby', 'Child', 'Youth', 'Middle Aged', 'Senior']].idxmax(axis=1)


In [24]:
#df_score

In [25]:
df_trait = df_score[['Gender', 'Race', 'Hair color', 'Hair Shape', 'Face Shape', 'Looking Age']]

In [26]:
df_trait

,Gender,Race,Hair color,Hair Shape,Face Shape,Looking Age
images (2).jpeg,Female,White,Brown Hair,Wavy Hair,Oval Face,Youth
images (4).jpeg,Female,White,Brown Hair,Wavy Hair,Oval Face,Youth
téléchargement (8).jpeg,Female,White,Blond Hair,Wavy Hair,Oval Face,Youth
images.jpeg,Female,White,Brown Hair,Wavy Hair,Oval Face,Youth
téléchargement (10).jpeg,Female,White,Brown Hair,Wavy Hair,Oval Face,Youth
images (1).jpeg,Female,White,Blond Hair,Wavy Hair,Oval Face,Youth
images (3).jpeg,Female,White,Brown Hair,Wavy Hair,Oval Face,Youth
téléchargement.jpeg,Female,White,Brown Hair,Wavy Hair,Oval Face,Youth
images (7).jpeg,Female,White,Brown Hair,Wavy Hair,Oval Face,Youth
images (5).jpeg,Female,White,Blond Hair,Wavy Hair,Oval Face,Youth


In [27]:
list_trait = df_trait[['Gender', 'Race', 'Hair color', 'Hair Shape', 'Face Shape', 'Looking Age']].value_counts()[:1].index.tolist()

In [28]:
list_trait

[('Female', 'White', 'Brown Hair', 'Wavy Hair', 'Oval Face', 'Youth')]